In [1]:
# reading spare ice netcdf (gzipped) files and outputting monthly files for totals 
#  and counts onto a chosen grid, which can then be read by another
#  python script
# dduncan, 20/12/17
# modifying spare-ice l2 code to do the same for GPROF L2 data, 30/12

#import netCDF4
#from mpl_toolkits.basemap import Basemap, cm
#from pyhdf.SD import SD, SDC
#import gzip
#import shutil
#import tempfile
import os
import h5py

In [2]:
#def open_netcdf(fname):
#    if fname.endswith(".gz"):
#        infile = gzip.open(fname, 'rb')
#        tmp = tempfile.NamedTemporaryFile(delete=False)
#        shutil.copyfileobj(infile, tmp)
#        infile.close()
#        tmp.close()
#        data = netCDF4.Dataset(tmp.name)
#        os.unlink(tmp.name)
#    else:
#        data = netCDF4.Dataset(fname)
#    return data

In [3]:
yr = '15'
dn = 'day' # new 2/1/18, day night selection (previous was day hardcoded)
#dirt = '/home/dudavid/Dendrite/Dendrite/SatData/SpareICE/avhrr_345_mhs_345_angles_tsurfcfsr_all_global_h/20'+yr+'/' 
dirt = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/GPM/AM2_l2/20'+yr+'/' 
months = sorted(os.listdir(dirt))
print(months)

startlat = 90.0 # abs value of NS boundaries
gsize = 2.5 # grid size in degree
nx, ny = int(360/gsize), int(startlat*2/gsize)
sk = 2 # skipping across scans to speed up initially


['1501', '1502', '1503', '1504', '1505', '1506', '1507', '1508', '1509', '1510', '1511', '1512']


In [10]:
for m in months[11:]:
    print('month: ',m)
    iwpg,iwpgc = np.zeros([ny,nx]), np.zeros([ny,nx]) # re-init for each month
    iwpall = []
    days = sorted(os.listdir(dirt+m+'/'))
    ndays = 0 #init
    for d in days:
        print(d)
        files = sorted(os.listdir(dirt+m+'/'+d))
        if len(files) != 0:  # in case no files in day dir
            ndays += 1
            for fix in files[:1]: #range(len(files)):
                file = dirt+m+'/'+d+'/'+fix
                f = h5py.File(file,'r')
                iw = f['S1/iceWaterPath']#[:,::sk]
                half = round(np.shape(iw)[0]/2)-1 # half of nscans, effectively
                # taking FIRST half of scans from AM2 should guarantee daytime obs only (asc orbit)
                if dn == 'night': h1,h2 = half, np.shape(iw)[0]
                if dn == 'day':   h1,h2 = 0, half
                iwt = iw[h1:h2,::sk] #,np.size(iw))
                iwp = np.reshape(iwt,np.size(iwt)) # make 1D
                #pop= np.reshape(np.array(f['S1/probabilityOfPrecip']),np.size(iw))
                lat= np.reshape(f['S1/Latitude'][h1:h2,::sk],np.size(iwp))
                lon= np.reshape(f['S1/Longitude'][h1:h2,::sk],np.size(iwp))
                #sct= f['S1/SCstatus/SClatitude'][h1:h2]#,np.size(iwp))
                #scd = [sct[x+1]-sct[x] for x in range(len(sct)-1)]
                #print(half,info(scd))
                #sfc= np.reshape(np.array(f['S1/surfaceTypeIndex']),np.size(iw))
                
                scrn = [iwp >= 0.0] #AVOIDs missing (negative) values
                #print(len(iwp),len(iwp[scrn]))
                if len(scrn) == 0: print('problem!')
        
                # calculate lat/lon indices for Xdeg grid (fixed 23/11):
                ladex = np.round((lat[scrn]+startlat-gsize*.5) / gsize)
                lodex = np.round((lon[scrn]+180.0-gsize*.5) / gsize)
                siwp = iwp[scrn] # >=0 values only!
                if len(siwp) > 0: 
                    scndscrn = [abs(lat[scrn]) <= 60]
                    iwpall.extend(siwp[scndscrn]) # just take 60NS points for histo
                    #print(np.max(siwp))#info(siwp))#,info(iwp))

                    ### commented out for histo/pdf calculation
                #for x in range(nx):
                #    if np.any(lodex == x):
                #        dexsub = np.where(lodex == x)
                #        subset = siwp[dexsub] 
                #        subsetla = ladex[dexsub]
                #        for y in range(ny):
                #            if np.any(subsetla == y):
                #                lolasubset = np.where(subsetla == y)
                #                iwpg[y,x] += subset[lolasubset].sum() # add up, get mean afterward
                #                iwpgc[y,x] += len(subset[lolasubset])
                                
    nd = str(ndays)
    # dataset name, res (x,y), time res of input, output arrays (grids, zonals), version?, # days
    #fin = 'grids/gprofAM2.'+dn+'.'+str(nx)+'x.'+str(ny)+'y.iwptot.cts.hist.v1.'+m+'.'+nd
    #if os.path.isfile(fin+'.tot.npy'):
    #    print(outfi,' already exists!')
    #else:
    #    np.save(fin+'.tot',iwpg)
    #    np.save(fin+'.cts',iwpgc)
        
    #create histogram of IWP
    binz = [2**x for x in range(-14,6)]
    binzz= [-.001,2**(-14)]
    hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
    histaz, bin_edges = np.histogram(np.array(iwpall),bins=binzz)
    fin = 'grids/gprofAM2.'+dn+'.60NS.histo.v2.'+m+'.'+nd # didnt run a year cuz
    #  'zero' gprof results for IWP are less than one in a 1e5 or so, totally negligible for plot
    if os.path.isfile(fin+'.npy'):
        print(fin+'.npy',' already exists!')
    else:
        outz = np.append(histaz,hista)
        print('out',outz)
        np.save(fin,outz)#hista)
                


month:  1512
151201
151202
151203
151204
151205
151206
151207
151208
151209
151210
151211
151212
151213
151214
151215
151216
151217
151218
151219
151220
151221
151222
151223
151224
151225
151226
151227
151228
151229
151230
151231
out [    121     539    2694 1128313 1523562 1145485 1407085 1455521 1170339
  661973  433008  283865  171973  138973  111131   66792   30924    2361
      15       1]


In [5]:
print(info(iwpall))

[0.0, 53.350182, 0.029639434, 0.1801248]


In [6]:
#b = [iwpall > 3.0]
#print(b)
#print(info(iwpall[b]))